# Analysis of neuron networks, as they change over time.

In [1]:
from analysis import analysis_utils as au
from analysis import resampling as rs
from analysis.graph_analysis_utils import neuron_network
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from scipy import stats
import seaborn as sns
import SigProc
import sys

In [2]:
%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

In [3]:
sns.set_style("darkgrid")

In [4]:
mouse_directory = os.path.join(os.path.expanduser("~"), "Hen_Lab/Mice/EPM")

if not os.path.exists(mouse_directory):
    print("The mouse directory does not exist", file=sys.stderr)

file_num = 0
raw_files = list()
for dir_name, subdir_list, file_list in os.walk(mouse_directory):
    for file_name in file_list:
        if file_name.endswith(".csv"):
            print("{}. full path of: {} is: {}".format(file_num, file_name, os.path.join(dir_name, file_name)))
            file_num += 1
            raw_files.append(os.path.join(dir_name, file_name))

0. full path of: behavior_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/behavior_drd87.csv
1. full path of: Raw_EPM_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/Raw_EPM_drd87.csv
2. full path of: Raw_EPM_drd73.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd73/Raw_EPM_drd73.csv
3. full path of: behavior_drd73.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd73/behavior_drd73.csv
4. full path of: Raw_EPM_drd77.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd77/Raw_EPM_drd77.csv
5. full path of: behavior_drd77.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd77/behavior_drd77.csv
6. full path of: behavior_drd46.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd46/behavior_drd46.csv
7. full path of: Raw_EPM_drd46.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd46/Raw_EPM_drd46.csv


In [5]:
data = pd.read_csv(raw_files[1], header=None)
_, AUC_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(data, 2, 0.5, 0.2, 10)

In [6]:
behavior_column_names = ["Trial_time", 'Recording_time', 'X_center', 'Y_center', 'Area', 'Areachange', 
                         'Elongation', 'Distance_moved', 'Velocity', 'Arena_centerpoint',
                         'Open1_centerpoint', 'Open2_centerpoint',
                         'Closed1_centerpoint', 'Closed2_centerpoint',
                         'OpenArms_centerpoint', 'ClosedArms_centerpoint', 'Result_1']

behavior_df = pd.read_csv(raw_files[0], header=None)
behavior_df.columns = behavior_column_names
behavior_df = au.downsample_dataframe(behavior_df, 3)

# Define what constitutes as a running frame
VELOCITY_CUTOFF = 4;

# Adds "Running_frames" column to the end of the behavior Dataframe 
behavior_df["Running_frames"] = np.where(behavior_df["Velocity"] > VELOCITY_CUTOFF, 1, 0)

# Rename all the columns from "neuron_x" --> "x". This makes the graphs neater by making sure the neuron names fit into the nodes
cell_transients_dataframe.columns = [i for i in range(1, len(cell_transients_dataframe.columns)+1)]

In [7]:
neuron_concated_behavior = AUC_dataframe.join(behavior_df, how="left")

In [8]:
def bin_by_time(dataframe, **kwargs):
    """Split dataframe by frequency
    
    Split up the time series data into specified time bins. 
    
    Args: 
        dataframe: DataFrame
        
            a pandas DataFrame of the signal data for 
            all the imaged neurons of a given animal activity.
        
        freq: string, optional
            
            If specified, this is the frequency by which the 
            passed DataFrame will be split. The default is 
            1Min.
    
    Returns: 
        time_bins: dictionary
            
            All of the DataFrames that encompasse the passed-in
            DataFrame.
    """
    
    # Create copy of the dataframe for a certain behavior
    time_binned_df = dataframe.copy()
    time_binned_df.reset_index(drop=True, inplace=True)
    
    # Add a column of the trial time in the form of time deltas
    x = pd.to_timedelta("0.1s")
    time_binned_df.loc[:, "TIME"] = pd.Series(x*i for i in (time_binned_df.index))
    
    # Group the dataframe by 1 minute intervals
    grouped = time_binned_df.set_index("TIME").groupby(pd.Grouper(freq=kwargs.get("freq", "1Min")))
    
    # Place each dataframe that contains the data for every 1 minute intervals into a dictionary
    time_bins = {}
    freq = 0
    for name, group in grouped:
        time_bins[freq] = grouped.get_group(name)
        freq += 1
        
    return time_bins

# We begin by plotting the networks, over time, for activity recorded in the **OpenArms**, and in the **ClosedArms**

## In accordance with Network neuroscience (Bassett & Sporns 2017), we plot the state of the network of neurons for DRD87 for the duration of the EPM experiment.

In [ ]:
time_binned_dfs = bin_by_time(cell_transients_dataframe, freq="1Min")

for i in range(len(time_binned_dfs)):
    open_graph = neuron_network(time_binned_dfs[i].reset_index(drop=True))
    open_graph.plot_network(figsize=(35,35), node_color="pink", node_size=1000)

## We get all of the indices in which DRD87 was marked as in the **OpenArms**, and we establish all the 1 minute intervals of that behavior. Namely, we create a separate dataframe for each minute of activity in the **OpenArms**.

## Then, we loop through all of the dataframes we created, and we plot their representations as graphs. Note: we reset (and drop the old) index prior to passing it to the `create_graph()` function.

In [ ]:
indices = neuron_concated_behavior.loc[neuron_concated_behavior["OpenArms_centerpoint"] != 0].index
time_binned_dfs = bin_by_time(cell_transients_dataframe.iloc[indices])

for i in range(len(time_binned_dfs)):
    open_graph = gau.create_graph(time_binned_dfs[i].reset_index(drop=True))
    gau.plot_cluster_graph(open_graph, figsize=(35,35), node_color="pink", node_size=1000)

## Now we repeat the same exact procedure, but for the **ClosedArms**

In [ ]:
indices = neuron_concated_behavior.loc[neuron_concated_behavior["ClosedArms_centerpoint"] != 0].index
time_binned_dfs = bin_by_time(cell_transients_dataframe.iloc[indices], freq="15s")

for i in range(len(time_binned_dfs)):
    open_graph = gau.create_graph(time_binned_dfs[i].reset_index(drop=True))
    gau.plot_cluster_graph(open_graph, figsize=(35,35), node_color="lightgreen", node_size=1000)

# Now, we plot the networks, over time, for *continous* activity, for certain behaviors. E.g., $5$ seconds of activity in the **OpenArms**

In [ ]:
def get_nodes_with_degree(graph):
    nodes = set()
    
    for node in graph.degree():
        if node[1] > 0:
            nodes.add(node[0])
            
    return nodes

In [9]:
import networkx as nx

def get_continuous_beh_graphs(neuron_concated_behavior, cell_transients_dataframe, behavior, **kwargs):
    num_of_sec = kwargs.get("seconds", 5)
    framerate = kwargs.get("framerate", 10)
    graphs = list()
    
    continuous_beh = dict()
    for row in neuron_concated_behavior.itertuples():
        if getattr(row, behavior) == 1:
            continuous_beh[row[0]] = 1
        else:
            if len(continuous_beh) >= num_of_sec*framerate:

                # Note the start frame, and the end frame for when the animal entered the area.
                begin = list(continuous_beh.keys())[0]
                end = list(continuous_beh.keys())[len(list(continuous_beh.keys()))-1]

                # time_binned_dfs = bin_by_time(cell_transients_dataframe.loc[begin:end], freq="1Min")
                graphs.append(neuron_network(cell_transients_dataframe.loc[begin:end]))

                continuous_beh.clear()
            else:
                continuous_beh.clear()
                
    return graphs

In [10]:
import plotly
import plotly.graph_objs as go

def plot_network_measures(y1, y2, x, **kwargs):
    trace1 = go.Scatter(
        x=x,
        y=y1,
        text=y1,
        textposition="auto",
        name="Open Arms",
        mode="lines+markers",
        marker=dict(color="rgb(255, 0, 0)"),
        opacity=0.6
    )
    trace2 = go.Scatter(
        x=x,
        y=y2,
        text=y2,
        textposition="auto",
        name="Closed Arms",
        mode="lines+markers",
        marker=dict(color="rgb(58,200,225)"),
        opacity=0.6
    )
    data = [trace1, trace2]
    layout = go.Layout(
        title = kwargs.get("title", "Title Goes Here"),
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [11]:
open_arms_graphs = get_continuous_beh_graphs(neuron_concated_behavior, cell_transients_dataframe, "OpenArms_centerpoint", seconds=3)
closed_arms_graphs = get_continuous_beh_graphs(neuron_concated_behavior, cell_transients_dataframe, "ClosedArms_centerpoint", seconds=3)

open_degree_centralities = list()
for graph in open_arms_graphs:
    open_degree_centralities.append(graph.compute_mean_degree_cent())

np.mean(open_degree_centralities)
    
closed_degree_centralities = list()
for graph in closed_arms_graphs:
    closed_degree_centralities.append(graph.compute_mean_degree_cent())

np.mean(closed_degree_centralities)

x_axis = max(len(open_degree_centralities), len(closed_degree_centralities))
plot_network_measures(open_degree_centralities, closed_degree_centralities, x_axis, title="DRD87 Mean Degree Centrality")

0.042436298190773894

0.04006820119352088

In [12]:
open_connection_densities = list()
for graph in open_arms_graphs:
    open_connection_densities.append(graph.compute_connection_density())

np.mean(open_connection_densities)
    
closed_connection_densities = list()
for graph in closed_arms_graphs:
    closed_connection_densities.append(graph.compute_connection_density())

np.mean(closed_connection_densities)

x_axis = max(len(open_connection_densities), len(closed_connection_densities))
plot_network_measures(open_connection_densities, closed_connection_densities, x_axis, title="DRD87 Connection Density")

0.042436298190773894

0.04006820119352089

In [13]:
open_measures = list()
for graph in open_arms_graphs:
    open_measures.append(graph.compute_max_clique_size())

np.mean(open_measures)
    
closed_measures = list()
for graph in closed_arms_graphs:
    closed_measures.append(graph.compute_max_clique_size())

np.mean(closed_measures)

x_axis = max(len(open_measures), len(closed_measures))
plot_network_measures(open_measures, closed_measures, x_axis, title="DRD87 Max Clique Size")

7.888888888888889

7.235294117647059

In [14]:
open_measures = list()
for graph in open_arms_graphs:
    open_measures.append(graph.compute_mean_clique_size())

np.mean(open_measures)
    
closed_measures = list()
for graph in closed_arms_graphs:
    closed_measures.append(graph.compute_mean_clique_size())

np.mean(closed_measures)

x_axis = max(len(open_measures), len(closed_measures))
plot_network_measures(open_measures, closed_measures, x_axis, title="DRD87 Mean Clique Size")

3.8865239849884046

3.362448365106171

In [15]:
open_measures = list()
for graph in open_arms_graphs:
    open_measures.append(nx.average_clustering(graph.graph))

np.mean(open_measures)
    
closed_measures = list()
for graph in closed_arms_graphs:
    closed_measures.append(nx.average_clustering(graph.graph))

np.mean(closed_measures)

x_axis = max(len(open_measures), len(closed_measures))
plot_network_measures(open_measures, closed_measures, x_axis, title="DRD87 Clustering Coefficient")

0.2922841169655456

0.2890326605101633

In [16]:
open_measures = list()
for graph in open_arms_graphs:
    open_measures.append(nx.local_efficiency(graph.graph))

np.mean(open_measures)
    
closed_measures = list()
for graph in closed_arms_graphs:
    closed_measures.append(nx.local_efficiency(graph.graph))

np.mean(closed_measures)

x_axis = max(len(open_measures), len(closed_measures))
plot_network_measures(open_measures, closed_measures, x_axis, title="DRD87 Local Efficiency")

0.3420874698289783

0.3892493462968492

In [17]:
open_measures = list()
for graph in open_arms_graphs:
    open_measures.append(nx.global_efficiency(graph.graph))

np.mean(open_measures)
    
closed_measures = list()
for graph in closed_arms_graphs:
    closed_measures.append(nx.global_efficiency(graph.graph))

np.mean(closed_measures)

x_axis = max(len(open_measures), len(closed_measures))
plot_network_measures(open_measures, closed_measures, x_axis, title="DRD87 Global Efficiency")

0.07024057610161624

0.08077074393269705